In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [ ]:
path = os.path.dirname(os.getcwd()) + "/data/"

In [ ]:
data = pd.read_csv(path + "telco.csv").iloc[:-1]

In [ ]:
desc = pd.read_excel(path + "field_descriptions.xlsx").set_index("Fields")

In [ ]:
data.select_dtypes(exclude ="object").describe()

## User Overview Analysis
* Start by identifying the top 10 handsets used by the customers.

* Then, identify the top 3 handset manufacturers

* Next, identify the top 5 handsets per top 3 handset manufacturer

* Make a short interpretation and recommendation to marketing teams

In [ ]:
new_data_col = [i.strip() for i in list(data.columns)]

In [ ]:
data.columns = new_data_col

In [ ]:
handset_data = data[['Handset Manufacturer', 'Handset Type',]]

In [ ]:
top10_handet = handset_data["Handset Type"][handset_data["Handset Type"] != "undefined"].value_counts()[:10]#.plot(kind = "bar")

In [ ]:
top3_manufacturers = handset_data["Handset Manufacturer"][handset_data["Handset Manufacturer"] != "undefined"].value_counts().head(3)

In [ ]:
handsets_by_top3_manufacturers = handset_data.set_index(keys= ["Handset Manufacturer"]).loc[top3_manufacturers.keys()]

In [ ]:
top5_of_top3_manufacturers_dict = dict()
for i in top3_manufacturers.keys():
    top5_of_top3_manufacturers_dict[i] = handsets_by_top3_manufacturers.loc[i, "Handset Type"].value_counts().head(5)

In [ ]:
top5_of_top3_manufacturers = pd.DataFrame.from_dict(top5_of_top3_manufacturers_dict, orient= "index").stack()
top5_of_top3_manufacturers.name = "count"
top5_of_top3_manufacturers = top5_of_top3_manufacturers.to_frame()

## Task 1.1
Aggregate per user the following information in the column  
- number of xDR sessions
- session duration
- the total download (DL) and upload (UL) data
- the total data volume (in Bytes) during this session for each application

In [ ]:
data["MSISDN/Number"] = data["MSISDN/Number"].fillna(-999).astype("int")

In [ ]:
no_of_xdr_sessions = data["MSISDN/Number"].fillna("median").value_counts()

In [ ]:
total_sessions_duration = (data[["MSISDN/Number", "Dur. (ms)"]]
                           .fillna("mean").groupby(["MSISDN/Number"])["Dur. (ms)"]
                           .sum()
                          )/(1000*60)

In [ ]:
total_dl_ul_data = (data[["MSISDN/Number", "Total DL (Bytes)", "Total UL (Bytes)"]]
                    .fillna("mean").groupby(["MSISDN/Number"])[["Total DL (Bytes)", "Total UL (Bytes)"]]
                    .sum()
                   )/(2**20)

In [ ]:
total_dl_ul_data["total_data"] = total_dl_ul_data.sum(1)

In [ ]:
apps = ['Social Media DL (Bytes)', 'Social Media UL (Bytes)',
       'Google DL (Bytes)', 'Google UL (Bytes)', 'Email DL (Bytes)',
       'Email UL (Bytes)', 'Youtube DL (Bytes)', 'Youtube UL (Bytes)',
       'Netflix DL (Bytes)', 'Netflix UL (Bytes)', 'Gaming DL (Bytes)',
       'Gaming UL (Bytes)', 'Other DL (Bytes)', 'Other UL (Bytes)',]

In [ ]:
total_dl_ul_data_per_app = (data[apps + ["MSISDN/Number"]].fillna("mean")
                            .groupby(["MSISDN/Number"])[apps].sum()
                           )/(2**20)

In [ ]:
total_data_per_app = pd.DataFrame()
count = 0
for i,j in zip(range(len(total_dl_ul_data_per_app.columns)), total_dl_ul_data_per_app.columns):
    if count != 1:
        total_data_per_app[j.split()[0]] = total_dl_ul_data_per_app.iloc[:, i] + total_dl_ul_data_per_app.iloc[:,i+1]
        count = (count + 1) % 2
    else: 
        count = (count + 1) % 2
        continue

In [ ]:
aggregates = (no_of_xdr_sessions.to_frame()
              .join(total_sessions_duration)
              .join(total_data_per_app)
              .join(total_dl_ul_data)
             )

In [ ]:
aggregates.index.name = "MSISDN"
aggregates.columns = ["tot number of session", "tot sessions duration (mins)", "social media data usage (GBs)",
                      "google data usage (MBs)","Email data usage (MBs)", "youtube data usage (MBs)", 
                      "netflix data usage (MBs)",  "gaming apps data usage (MBs)", "other apps data usage (MBs)", 
                     "tot downloaded data (MBs)", "tot uploaded data (MBs)", "tot data usage (MBs)",]
aggregates = aggregates.drop(index = -999)

In [ ]:
aggregates.iloc[1:].describe().style#.set_properties(**{"width": "100px"})

## Task 1.2

○	Describe all relevant variables and associated data types (slide). 

○	Analyze the basic metrics (mean, median, etc) in the Dataset (explain) & their importance for the global objective.

○	Conduct a Non-Graphical Univariate Analysis by computing dispersion parameters for each quantitative variable and providing useful interpretation. 

○	Conduct a Graphical Univariate Analysis by identifying the most suitable plotting options for each variable and interpreting your findings.

○	Bivariate Analysis – explore the relationship between each application & the total DL+UL data using appropriate methods and interpret your findings. 

○	Variable transformations – segment the users into the top five decile classes based on the total duration for all sessions and compute the total data (DL+UL) per decile class. 

○	Correlation Analysis – compute a correlation matrix for the following variables and interpret your findings: Social Media data, Google data, Email data, Youtube data, Netflix data, Gaming data, Other data 

○	Dimensionality Reduction – perform a principal component analysis to reduce the dimensions of your data and provide a useful interpretation of the results (Provide your interpretation in four (4) bullet points-maximum). 


In [ ]:
aggregates.dtypes

In [ ]:
ss = aggregates.describe().loc[["mean", "min", "50%", "75%", "max"]].T

In [ ]:
ss.style.set_precision(2).set_properties(**{"width": "100px", "color": "blue"})

In [ ]:
aggregates.iloc[:,:1].boxplot(figsize= (20,10), whis= (0,100))

In [ ]:
aggregates.iloc[:,1:2].boxplot(figsize= (20,10), whis= (0,100))

In [ ]:
aggregates.iloc[:,2:7].boxplot(figsize= (20,10), whis= (0,100))

In [ ]:
aggregates.iloc[:,7:12].boxplot(figsize= (20,10), whis= (0,100))